----
# **`B`i`N`a`N`a** - **`{Binance Trading Bot}`**
----

## **#1 - Import and initialization of necessary objects**
- ### Connect to the binance API - **`Required`**

In [1]:
""" #1 - CODE >>>>>>
"""
import sys
import plotly.graph_objects as go
import pandas as pd
from binance.client import Client
from time import sleep
from datetime import datetime

api_key = "GTX9gXFumeoGthX0eYY3tv3jXnt8yq6BfBkyQ2Tc2tKiqjEoGmiQrcMluw6Qa7os"
api_secret = "ANnbpbGVhwYEBcTyLsbmBRj9rTyg1RwYj4NaX6RZS63DbSSiCcNRbOJdAjJaFvkZ"

# CONNECT CLIENT >>
client = Client(api_key, api_secret, testnet=False)
if client is not None:
    print(f"\nClient: [client] successfy connected to [{client.API_URL}]")
    print(f"Server Time: {datetime.fromtimestamp(int(client.get_server_time().get('serverTime')/1000))}")
    print(f"API: {client.PUBLIC_API_VERSION}")
else:
    print("Unable to connect to Binance API server")


Client: [client] successfy connected to [https://api.binance.com/api]
Server Time: 2021-11-16 09:14:06
API: v1


----
##  **#2 - Collect Data**
- ### Get and structure data in Pandas DataFrame() - **`Required`**

In [3]:
""" #2 - RUN DATA COLLECTOR >>>>>>
"""
#TODO lage et lite api av progressbar funksjonene

coins = ['BTC', 'ETH', 'XRP']       # Specify which currency to include in data collection

# ------------------------------>
# MAIN REQUEST LOOP ---------->
for i, coin in enumerate(coins):
    if not i: sys.stdout.write("[%-20s] %d%%" % ('='*0, 30*i))  #Progress bar
    print(f' Gathering {coin}...')
    start_str = 'Nov 1, 2021' #'2021-10-01' #'Oct 1, 2021' 
    end_str = 'Nov 14, 2021'#'2021-11-01' #'Nov 1, 2021'
    klines = client.get_historical_klines(
        symbol=f'{coin}USDT',
        interval=client.KLINE_INTERVAL_1HOUR,
        start_str=start_str,
        end_str=end_str)
    
    # PROGRESS BAR PROGRESSING ---->
    currentbar = (i+1)*7
    lastbar = i*7
    barPercent = float(currentbar/21)
    for nbar in range(lastbar, currentbar):
        sys.stdout.write('\r')
        sys.stdout.write("[%-20s] %d%%" % ('='*nbar, (nbar/currentbar)*100*barPercent))
        sys.stdout.flush()
        sleep(0.04)

    # Create column name list for corresponding data
    cols = ['OpenTime',
            f'{coin}/USDT_Open',
            f'{coin}/USDT_High',
            f'{coin}/USDT_Low',
            f'{coin}/USDT_Close',
            f'{coin}/USDT_Volume',
            'CloseTime',
            f'{coin}-QuoteAssetVolume',
            f'{coin}-NumberOfTrades',
            f'{coin}-TBBAV',
            f'{coin}-TBQAV',
            f'{coin}-ignore',]
    coin_df = pd.DataFrame(klines,columns=cols)
    
    # Merge data to a single DataFrame
    df = coin_df if i <= 0 else pd.merge(df, coin_df, how='inner', on=['OpenTime', 'CloseTime'])

    # Confirm download success and update progress bar
    if i >= len(coins)-1:
        sys.stdout.write(f" Dataset downloaded successfully!\n")
    
    # Prevent API max request
    sleep(1)

# Convert raw timedata to datedime format
df['OpenTime'] = [datetime.fromtimestamp(ts / 1000) for ts in df['OpenTime']]
df['CloseTime'] = [datetime.fromtimestamp(ts / 1000) for ts in df['CloseTime']]
for col in df.columns:
    if not 'Time' in col:
        df[col] = df[col].astype(float)

# Progress bar >    # TODO - Animate progressbar at 100%   
if i >= len(coins)-1:       
    sys.stdout.write(f"[====================] {100}% DataFrame:'df' ready for deployment!")
else:
    sys.stdout.write('\r')
    sys.stdout.write("[%-20s] %d%%" % ('='*(i+1)*7, 30*(i+1)))
    sys.stdout.flush()


[                    ] 0% Gathering BTC...
[======              ] 28% Gathering ETH...
[=============       ] 61% Gathering XRP...
[====================] 95% Dataset downloaded successfully!
[====================] 100% DataFrame:'df' ready for deployment!

----
## **#3 - Plot Data**
### **showChart()** - Plot dataframe with Plotly - **`Required`**

In [4]:
## Plot
def showChart():
    for coin in coins:
        fig = go.Figure(data=[go.Candlestick(x=df['OpenTime'],
                                            open=df[f'{coin}/USDT_Open'],
                                            high=df[f'{coin}/USDT_High'],
                                            low=df[f'{coin}/USDT_Low'],
                                            close=df[f'{coin}/USDT_Close']
                                            )])
        fig.show()
        go.Figure.add_indicator()

----
## **#4 - Indicators**
- ### Initialize indicators - **`Required`**

In [6]:
# Simple Moving Average
def sma(data, window):
   return(data.rolling(window=window).mean())

# Bollinger Bands
def bollinger_band(data, sma, window, nstd):
    std = data.rolling(window = window).std()
    upper_band = sma + std * nstd
    lower_band = sma - std * nstd
    
    return upper_band, lower_band

### Add indicators to DataFrame -> - **`Required`**

In [ ]:
nstd = 3
sma(df['BTC/USDT_Open'], window=20)



for coin in coins:
    df[f'{coin}_sma'] = sma(df[f'{coin}/USDT_Open'], 20)
    df[f'{coin}_upper_band'], df[f'{coin}_lower_band'] = bollinger_band(df[f'{coin}/USDT_Open'], df[f'{coin}_sma'], 20, nstd)
    
    
df.dropna(inplace=True)

----
## **#5** - **`(CLASSES)`**

### **`TradingEnv`** - Virtual trading environment for testing - **`Required`**

In [8]:
class TradingEnv:
    # Initialize virtual trading account ->
    def __init__(self, balance_amount, balance_unit, trading_fee_multiplier):
        self.balance_amount = balance_amount                    # Account balance
        self.balance_unit = balance_unit                        # Display currency
        self.buys = []                                          # Buy history
        self.sells = []                                         # Sell history
        self.trading_fee_multiplier = trading_fee_multiplier    # Fee multiplier [VIP level 0, paying fees with 'BNB']
        
        
    # Virtual BUY ->
    def buy(self, symbol, buy_price, time):
        self.balance_amount = (self.balance_amount / buy_price) * self.trading_fee_multiplier   # 1. Sets the balance of the bought asset based on buyprice
        self.balance_unit = symbol                                                              # 2. The currency unit of the bougt asset 
        self.buys.append([symbol, time, buy_price])                                             # 3. Save buy action in a list
        
    # Virtual SELL >
    def sell(self, sell_price, time):
        self.balance_amount = self.balance_amount * sell_price * self.trading_fee_multiplier    # 1. Sets the account balance after sell
        self.sells.append( [self.balance_unit, time, sell_price] )                              # 2. Save sell action in a list
        self.balance_unit = 'USDT'                                                              # 3. Sets the balance unit to 'USDT' for account balance

### **`loadData()`** - Holds Volume, Price and Technical Data on demand


In [15]:
class loadData:
    """ Holds the Volume, Price, and Technical Data for the
        chosen indicators in a DataFrame

        Args:
            client (object): Binance API client
            pair (str): e.g 'BTCUSDT'
            interval (str): e.g '1m'
            start_str (str): Time in milliseconds/1000 .. not done <
            end_str (str): optional: end time of datasampling in millisec
            indicators (bool, [optional]): returns DataFrame of technicals if True. Defaults to False.
    """
    def __init__(self, client, pair, interval, start_str, end_str, indicators=False):
        client = self.client
        self.pair = pair
        self.start_str = start_str
        self.end_str = end_str
        self.indicators = indicators if indicators else False
        
    def priceFrame(self): #pair, interval, start_str, end_str):
        """Generates a simple DataFrame of the main price data

            Args:
                pair ([type]): [description]
                interval ([type]): [description]
                start_str ([type]): [description]
                end_str ([type]): [description]
        """
        print(self.pair, self.interval, self.start_str, self.end_str, "\n")
        print(self.client.get_symbol_ticker(self.pair))
        return None
        


----
## **#6 - Backtesting with virtual trading environement**

In [ ]:
showChart()

- ### Run strategy - **`Required`**

In [10]:
env = TradingEnv(balance_amount=100, balance_unit='USDT', trading_fee_multiplier=0.99925)

# The buy/sell conditional
for i in range(len(df)):
    if env.balance_unit == 'USDT': # --> BUY
        for coin in coins:
            if df[f'{coin}/USDT_Low'].iloc[i] < df[f'{coin}_lower_band'].iloc[i]: # --> BUY SIGNAL
                env.buy(coin, df[f'{coin}_lower_band'].iloc[i], df['OpenTime'].iloc[i])
                break

    if env.balance_unit != 'USDT': # --> SELL
        if df[f'{env.balance_unit}/USDT_High'].iloc[i] > df[f'{env.balance_unit}_upper_band'].iloc[i]: # --> SELL SIGNAL
            env.sell(df[f'{env.balance_unit}_upper_band'].iloc[i], df['OpenTime'].iloc[i])

if env.balance_unit != 'USDT':
    env.sell(df[f'{env.balance_unit}_upper_band'].iloc[-1], df['OpenTime'].iloc[-1])

In [14]:
print(f'num buys: {len(env.buys)}',
      f'\nnum sells: {len(env.sells)}',
      f'\nending balance: {env.balance_amount:.2f} {env.balance_unit}')

num buys: 4 
num sells: 4 
ending balance: 108.06 USDT


---

----
----
# Temp Functions >>>

## find outliers

In [ ]:
# Find outliers
from statistics import stdev

stdev = stdev(df['BTC/USDT_High'])

for i, val in enumerate(df['BTC/USDT_High']):
    lastval = df['BTC/USDT_High'].iloc[i-1]
    if abs(val-lastval) > stdev*12:
        print(f"{val} @{i} exceeding stdev")

In [ ]:
# PROGRESS BAR
from time import sleep
import sys
for i in range(21):
    sys.stdout.write('\r')
    # the exact output we're looking for:
    sys.stdout.write("[%-20s] %d%%" % ('='*i,5*i))
    sys.stdout.flush()
    sleep(0.25)